In [1]:
# Parameters
cell_type = "ILC"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/38 [00:00<?, ?it/s]

  3%|▎         | 1/38 [00:03<02:21,  3.83s/it]

  5%|▌         | 2/38 [00:07<02:21,  3.92s/it]

  8%|▊         | 3/38 [00:11<02:15,  3.86s/it]

 11%|█         | 4/38 [00:16<02:31,  4.46s/it]

...

 92%|█████████▏| 35/38 [02:16<00:09,  3.31s/it]

 95%|█████████▍| 36/38 [02:19<00:06,  3.30s/it]

 97%|█████████▋| 37/38 [02:23<00:03,  3.28s/it]

100%|██████████| 38/38 [02:25<00:00,  2.97s/it]

100%|██████████| 38/38 [02:25<00:00,  3.82s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/817 [00:00<?, ?it/s]

  1%|          | 9/817 [00:00<00:09, 89.62it/s]

  2%|▏         | 18/817 [00:00<00:12, 66.23it/s]

  4%|▍         | 36/817 [00:00<00:07, 108.81it/s]

  6%|▌         | 48/817 [00:00<00:07, 109.28it/s]

...

 95%|█████████▌| 778/817 [00:07<00:00, 82.40it/s]

 96%|█████████▋| 787/817 [00:07<00:00, 74.43it/s]

 98%|█████████▊| 800/817 [00:07<00:00, 85.86it/s]

 99%|█████████▉| 809/817 [00:07<00:00, 85.25it/s]

100%|██████████| 817/817 [00:07<00:00, 107.73it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:12,  1.54it/s]

 10%|█         | 2/20 [00:01<00:16,  1.06it/s]

 15%|█▌        | 3/20 [00:03<00:19,  1.13s/it]

 20%|██        | 4/20 [00:04<00:17,  1.12s/it]

...

 85%|████████▌ | 17/20 [00:22<00:04,  1.63s/it]

 90%|█████████ | 18/20 [00:24<00:03,  1.68s/it]

 95%|█████████▌| 19/20 [00:25<00:01,  1.60s/it]

100%|██████████| 20/20 [00:27<00:00,  1.62s/it]

100%|██████████| 20/20 [00:27<00:00,  1.36s/it]